In [1]:
from nuscenes.nuscenes import NuScenes
import json
import os

In [15]:
USE_RESULT_JSON = True
SCENE_INDEX = 0
FPS = 1
NUSCENES_VERSION = "v1.0-mini"
DATALOADER_TYPE = "nuscenes_loader"
RESULTS_ROOT = "../result/custom/"
DATA_ROOT = "/home/captainlevi/Documents/UCSD_lab_project/AVL/data/nuScenes_dataset/nuScenes/"

In [16]:
output_folder = os.path.join(RESULTS_ROOT, DATALOADER_TYPE, NUSCENES_VERSION, str(SCENE_INDEX))
data_path = os.path.join(DATA_ROOT, NUSCENES_VERSION)
if USE_RESULT_JSON == True:
    detector_json_file = os.path.join(output_folder, "results.json")

In [17]:
nusc = NuScenes(version=NUSCENES_VERSION, dataroot=data_path, verbose=True)
print("Using NuScenes dataset version: ", NUSCENES_VERSION)
# nuscExp = NuScenesExplorer(nusc)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.164 seconds.
Reverse indexing ...
Done reverse indexing in 0.0 seconds.
Using NuScenes dataset version:  v1.0-mini


**GET ALL ANNOTATIONS OF A SCENE**

In [18]:
def get_scene_annotations_nuscenes(scene_index):
    my_scene = nusc.scene[scene_index]
    first_sample_token = my_scene['first_sample_token']

    current_sample_token = first_sample_token
    result_dict = {}
    scene_sample_tokens = []

    while True:
        scene_sample_tokens.append(current_sample_token)
        current_sample = nusc.get('sample', current_sample_token)
        annotations = current_sample['anns']

        sample_annotations = []
        for annotation_token in annotations:
            annotation_metadata = nusc.get('sample_annotation', annotation_token)
            annotation_metadata["tracking_id"] = 1
            sample_annotations.append(annotation_metadata)
        result_dict[current_sample_token] = sample_annotations
        
        next_sample_token = current_sample['next']
        if next_sample_token == '':
            break
        current_sample_token = next_sample_token

    return result_dict, scene_sample_tokens

In [19]:
def get_scene_annotations_detector(detector_json_file):
    with open(detector_json_file) as f:
        results = json.load(f)

    scene_sample_tokens = list(results["results"].keys())

    return results["results"], scene_sample_tokens

In [20]:
if USE_RESULT_JSON == False:
    result_dict, scene_sample_tokens = get_scene_annotations_nuscenes(SCENE_INDEX)
else:
    result_dict, scene_sample_tokens = get_scene_annotations_detector(detector_json_file)

**VISUALIZE TRACKING 3D BOUNDING BOXES**

In [21]:
from nuscenes_utils import render_sample_data_custom

In [23]:
sample_token = scene_sample_tokens[2]

annotations_list = result_dict[sample_token]
sample_data_token = nusc.get('sample', sample_token)['data']['CAM_FRONT']

results_out_path_folder = "/home/captainlevi/Documents/UCSD_lab_project/AVL/MOT/codes/Poly-MOT/result/custom/"

render_sample_data_custom(sample_data_token, annotations_list, nusc, 
                          out_path = os.path.join(output_folder, 'sample_image.png'), verbose = True)

**MAKE SCENE VIDEO**

In [24]:
from vis_utils import scene_samples_render, generate_video

In [25]:
if USE_RESULT_JSON == False:
    images_folder = os.path.join(RESULTS_ROOT, "no_tracking", NUSCENES_VERSION, str(SCENE_INDEX), 'images')
    video_folder = os.path.join(RESULTS_ROOT, "no_tracking", NUSCENES_VERSION, str(SCENE_INDEX), 'videos')
else:
    images_folder = os.path.join(output_folder, 'images')
    video_folder = os.path.join(output_folder, 'videos')

if not os.path.exists(images_folder):
    os.makedirs(images_folder)
if not os.path.exists(video_folder):
    os.makedirs(video_folder)

In [26]:
# render images with bounding boxes for all samples
img_size, img_array = scene_samples_render(scene_sample_tokens, result_dict, nusc, images_folder)

100%|██████████| 39/39 [00:00<00:00, 41.14it/s]


In [27]:
# write video in mp4
video_path = os.path.join(video_folder, str(SCENE_INDEX) + '.mp4')
generate_video(img_array, video_path, FPS, img_size)

100%|██████████| 39/39 [00:00<00:00, 226.54it/s]


: 